In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import scipy
import numpy

# Reading in the data:

In [3]:
%%time
options = pd.read_csv("jpx-tokyo-stock-exchange-prediction/train_files/options.csv")

<timed exec>:1: DtypeWarning: Columns (7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 4.18 s, sys: 507 ms, total: 4.69 s
Wall time: 4.69 s


In [4]:
%%time
stock_prices = pd.read_csv("jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv")

CPU times: user 1.3 s, sys: 50.9 ms, total: 1.35 s
Wall time: 1.35 s


In [5]:
options.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3658100 entries, 0 to 3658099
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   DateCode             object 
 1   Date                 object 
 2   OptionsCode          int64  
 3   WholeDayOpen         float64
 4   WholeDayHigh         float64
 5   WholeDayLow          float64
 6   WholeDayClose        float64
 7   NightSessionOpen     object 
 8   NightSessionHigh     object 
 9   NightSessionLow      object 
 10  NightSessionClose    object 
 11  DaySessionOpen       float64
 12  DaySessionHigh       float64
 13  DaySessionLow        float64
 14  DaySessionClose      float64
 15  TradingVolume        int64  
 16  OpenInterest         int64  
 17  TradingValue         int64  
 18  ContractMonth        int64  
 19  StrikePrice          float64
 20  DaySessionVolume     int64  
 21  Putcall              float64
 22  LastTradingDay       float64
 23  SpecialQuotationDay  float64
 24

In [6]:
options[["DateCode", "Date", "OptionsCode", "WholeDayClose", "StrikePrice", "LastTradingDay"]]

,DateCode,Date,OptionsCode,WholeDayClose,StrikePrice,LastTradingDay
0,20170104_132010018,2017-01-04,132010018,480.0,20000.0,20170112.0
1,20170104_132010118,2017-01-04,132010118,0.0,20125.0,20170112.0
2,20170104_132010218,2017-01-04,132010218,0.0,20250.0,20170112.0
3,20170104_132010318,2017-01-04,132010318,0.0,20375.0,20170112.0
4,20170104_132010518,2017-01-04,132010518,0.0,20500.0,20170112.0
...,...,...,...,...,...,...
3658095,20211203_199244018,2021-12-03,199244018,0.0,44000.0,20241212.0
3658096,20211203_199245018,2021-12-03,199245018,0.0,35000.0,20241212.0
3658097,20211203_199246018,2021-12-03,199246018,0.0,36000.0,20241212.0
3658098,20211203_199247018,2021-12-03,199247018,0.0,37000.0,20241212.0


In [7]:
stock_prices.head()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026


# Exploring the data

In [8]:
stock_prices.Target.max()

1.119512195121951

# Processing the data

In [9]:
# Calculates the delta of the option using the Black Scholes Option Pricing Model.
def calculate_delta_BS(S, K, r, sigma, t, option_type):
    if option_type == "C":
        return scipy.stats.norm.cdf((np.log(S / K) + (r + 0.5 * sigma ** 2) * t) / (sigma * np.sqrt(t)))
    elif option_type == "P":
        return scipy.stats.norm.cdf((np.log(S / K) + (r + 0.5 * sigma ** 2) * t) / (sigma * np.sqrt(t))) - 1

In [10]:
def parseCode(row):
    code = str(row["OptionsCode"])
    return int(code[:4])


options["SecuritiesCode"] = options.apply(parseCode, axis=1)

In [11]:
options["SecuritiesCode"]

0          1320
1          1320
2          1320
3          1320
4          1320
           ... 
3658095    1992
3658096    1992
3658097    1992
3658098    1992
3658099    1992
Name: SecuritiesCode, Length: 3658100, dtype: int64

### Merging Option prices and Stock Price data left join

In [12]:
new_df = options.merge(stock_prices, how="inner", left_on=["SecuritiesCode", "Date"], right_on=["SecuritiesCode", "Date"])

In [13]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 612451 entries, 0 to 612450
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   DateCode             612451 non-null  object 
 1   Date                 612451 non-null  object 
 2   OptionsCode          612451 non-null  int64  
 3   WholeDayOpen         612451 non-null  float64
 4   WholeDayHigh         612451 non-null  float64
 5   WholeDayLow          612451 non-null  float64
 6   WholeDayClose        612451 non-null  float64
 7   NightSessionOpen     612451 non-null  object 
 8   NightSessionHigh     612451 non-null  object 
 9   NightSessionLow      612451 non-null  object 
 10  NightSessionClose    612451 non-null  object 
 11  DaySessionOpen       612451 non-null  float64
 12  DaySessionHigh       612451 non-null  float64
 13  DaySessionLow        612451 non-null  float64
 14  DaySessionClose      612451 non-null  float64
 15  TradingVolume    